In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot
import seaborn as sns
# Too see full dataframe...
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json # library to handle JSON files
#! pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
! pip install folium
import folium # map rendering library

print('Libraries imported.')
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
# First We have to locate the file path and changed accordingly
import os
os.getcwd()
#print(os.listdir("C:/Users/jaya/Documents/data"))



Libraries imported.


'/home/dsxuser/work'

In [17]:
# Link To Extract
path='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Read File
df_wiki=pd.read_html(path)
#Check the type
type(df_wiki)
# Call the position where the table is stored
neighborhood=df_wiki[0]
# Rename the Columns
neighborhood.rename(columns={'Postal code':'Postcode'},inplace=True)
neighborhood.rename(columns={0:'Postcode', 1: 'Borough', 2: 'Neighborhood'}, inplace=True)
neighborhood.rename(columns={0:'Postcode', 1: 'Borough', 2: 'Neighborhood'}, inplace=True)
# Eliminate the first row
neighborhood=neighborhood.drop([0])
# Eliminate "Not assigned", categorical values from "Borough" Column
neighborhood=neighborhood[neighborhood.Borough !='Not assigned']
# Making DataFrame
neighborhood=pd.DataFrame(neighborhood)
# Merging rows with same Postcode
neighborhood.set_index(['Postcode','Borough'],inplace=True)
merge_result = neighborhood.groupby(level=['Postcode','Borough'], sort=False).agg( ','.join)
# Setting the index
serial_wise=merge_result.reset_index()
# Assign the 'Borough' column value to 'Neighborhood' where 'Not assigned' occurs
serial_wise.loc[4, 'Neighborhood']='Queen\'s Park'
# Saving the file for future use!
serial_wise.to_excel('wikipedia_table.xls')
# Showing the Data Frame
df=pd.DataFrame(serial_wise)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park


In [18]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_0c13420e5d864f6e95b2a51e1b8dfcf6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='rbX1H1aUtheOp3Njb1IWGAOz0qsIxo6kXprdu3PPszQB',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_0c13420e5d864f6e95b2a51e1b8dfcf6.get_object(Bucket='week5finalcapstone-donotdelete-pr-wlietjj282xb54',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df1 = pd.read_csv(body)
df1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
#Geographical Coordinates
#df1=pd.read_csv("Geospatial_Coordinates.csv")
# Change the Postal Code to Postcode
df1.rename(columns={'Postal Code':'Postcode'},inplace=True)
#Cancatenation
frames=[df,df1]
frames=pd.concat(frames, axis=1, sort=False)
# Merging the two columns on 'Postcode'
merge_columns=pd.merge(df, df1, left_on='Postcode', right_on='Postcode')
# Save the Data Frame
merge_columns.to_csv('neigbors_geographical.csv')
merge_columns.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [20]:
#Sorting
# set index for only Downtown Toronto
downtown_toronto_data = merge_columns[merge_columns['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
 #eliminate 'Postcode' column
#downtown_toronto_data=downtown_toronto_data.drop(['Postcode'], axis=1)
downtown_toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


# Now we will move towards New York Boroughs. We select "Manhattan" as a Borough and anylze its neighborhoods later¶

In [23]:

body = client_0c13420e5d864f6e95b2a51e1b8dfcf6.get_object(Bucket='week5finalcapstone-donotdelete-pr-wlietjj282xb54',Key='newyork.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
neighborhoods= pd.read_csv(body)
neighborhoods.head()


,Unnamed: 0,Borough,Neighborhood,Latitude,Longitude
0,0,Bronx,Wakefield,40.894705,-73.847201
1,1,Bronx,Co-op City,40.874294,-73.829939
2,2,Bronx,Eastchester,40.887556,-73.827806
3,3,Bronx,Fieldston,40.895437,-73.905643
4,4,Bronx,Riverdale,40.890834,-73.912585


In [25]:
#neighborhoods=pd.read_csv("C:/Users/jaya/Documents/data/newyork.csv", index_col=0)
# And make sure that the dataset has all 5 boroughs and 306 neighborhoods.
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

neighborhoods.head()

The dataframe has 5 boroughs and 306 neighborhoods.


,Unnamed: 0,Borough,Neighborhood,Latitude,Longitude
0,0,Bronx,Wakefield,40.894705,-73.847201
1,1,Bronx,Co-op City,40.874294,-73.829939
2,2,Bronx,Eastchester,40.887556,-73.827806
3,3,Bronx,Fieldston,40.895437,-73.905643
4,4,Bronx,Riverdale,40.890834,-73.912585


In [26]:
#reating new Dataframe manhattan_data
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Unnamed: 0,Borough,Neighborhood,Latitude,Longitude
0,6,Manhattan,Marble Hill,40.876551,-73.910660
1,100,Manhattan,Chinatown,40.715618,-73.994279
2,101,Manhattan,Washington Heights,40.851903,-73.936900
3,102,Manhattan,Inwood,40.867684,-73.921210
4,103,Manhattan,Hamilton Heights,40.823604,-73.949688


## Foursquare API

In [27]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'YEOLYRVR5YBQRY4TPWQ4YAPJVNSRFBM2BBX1GTPABAJJV55C' # your Foursquare ID
CLIENT_SECRET = 'U1IFTZVYIEIG3U0KRKNJTGAC25UU0VEHOR01GDZVWFM4D3KK' # your Foursquare Secret
VERSION = '20180604'
limit = 20
print('Your credentails:')
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:'+ CLIENT_SECRET)

Your credentails:
CLIENT_ID:YEOLYRVR5YBQRY4TPWQ4YAPJVNSRFBM2BBX1GTPABAJJV55C
CLIENT_SECRET:U1IFTZVYIEIG3U0KRKNJTGAC25UU0VEHOR01GDZVWFM4D3KK


In [28]:
# get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

Downtown Toronto latitude 43.6563221 & longitude -79.3809161


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [29]:
#Let's get the geographical coordinates of Manhattan.
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


# VISUALIZATION
We visualize the data many times at different stages. 
In the beginning, we visualize the selected borough neighborhoods so that we can get an idea or 
confirmation regarding the coordinates of that Borough. The second time after clustered the neighborhoods, 
we visualize the clusters to name them. 
Assigning the names are very important because it can identify the areas or specific places in each cluster.

# Before Clustering)
# Downtown Toronto¶

In [30]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [31]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

# Manhattan¶

In [32]:
#let's visualizat Manhattan the neighborhoods in it.
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [33]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

grouping = plugins.MarkerCluster().add_to(map_manhattan)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(grouping)  
    
map_manhattan

# ANALYSIS
We analyze both boroughs neighborhoods through one hot encoding (giving ‘1’ if a venue category is there, and ‘0’ in case of venue category is not there). On the basis of one hot encoding, we calculate mean of the frequency of occurrence of each category and picked top ten venues on that basis for each neighborhood. 
It means the top venues are showing the foot traffic or the more visited places

In [34]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
#Write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'],
                                  )

Regent Park / Harbourfront
Queen's Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Rosedale
Stn A PO Boxes
St. James Town / Cabbagetown
First Canadian Place / Underground city
Church and Wellesley


In [36]:
#Let's check the size of the resulting dataframe
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(356, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [37]:
# Let's check how many venues were returned for each neighborhood
downtown_toronto_venues.groupby('Neighborhood').count()



,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,20,20,20,20,20,20
Christie,16,16,16,16,16,16
Church and Wellesley,20,20,20,20,20,20
Commerce Court / Victoria Hotel,20,20,20,20,20,20
First Canadian Place / Underground city,20,20,20,20,20,20
"Garden District, Ryerson",20,20,20,20,20,20
Harbourfront East / Union Station / Toronto Islands,20,20,20,20,20,20


In [38]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categoriesa.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 122 uniques categoriesa.


# Analyzing Each Neighborhood


In [39]:
#one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Electronics Store,Farmers Market,Fish Market,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gastropub,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
 #Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [41]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.10
1         Cheese Shop  0.05
2      Breakfast Spot  0.05
3              Museum  0.05
4            Fountain  0.05


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.30
1           Ramen Restaurant  0.05
2                  Gastropub  0.05
3  Middle Eastern Restaurant  0.05
4                        Spa  0.05


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3     Baby Store  0.06
4      Nightclub  0.06


----Church and Wellesley----
               venue  freq
0   Ramen Restaurant  0.05
1  Martial Arts Dojo  0.05
2               Park  0.0

In [42]:
 #Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Bistro,Farmers Market,Bakery,Liquor Store,Breakfast Spot,Beer Bar,Concert Hall,Coffee Shop,Cocktail Bar
1,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Bar,Boat or Ferry,Sculpture Garden,Airport Gate
2,Central Bay Street,Coffee Shop,Gastropub,Ice Cream Shop,Café,Park,Modern European Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Ramen Restaurant,Sandwich Place
3,Christie,Grocery Store,Café,Park,Candy Store,Coffee Shop,Diner,Baby Store,Italian Restaurant,Restaurant,Nightclub
4,Church and Wellesley,Park,Diner,Juice Bar,Restaurant,Beer Bar,Ramen Restaurant,Bookstore,Italian Restaurant,Creperie,Dance Studio
5,Commerce Court / Victoria Hotel,Café,Coffee Shop,Bakery,Gastropub,Gym / Fitness Center,Ice Cream Shop,Japanese Restaurant,Deli / Bodega,Museum,Pub
6,First Canadian Place / Underground city,Café,Coffee Shop,Restaurant,Pizza Place,Bookstore,Deli / Bodega,Bakery,Steakhouse,Pub,Gym
7,"Garden District, Ryerson",Café,Shopping Mall,Steakhouse,Burrito Place,Mexican Restaurant,Clothing Store,Coffee Shop,Plaza,Ramen Restaurant,Restaurant
8,Harbourfront East / Union Station / Toronto Is...,Park,Hotel,Plaza,Café,Performing Arts Venue,Bubble Tea Shop,Lake,Salad Place,Japanese Restaurant,Skating Rink
9,Kensington Market / Chinatown / Grange Park,Café,Vietnamese Restaurant,Mexican Restaurant,Wine Bar,Bakery,Fish Market,Dessert Shop,Coffee Shop,Cocktail Bar,Organic Grocery


# Clustering Neighborhoods


In [44]:
 #set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 0, 3, 1, 3, 3, 3, 1, 3], dtype=int32)

In [45]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_toronto_merged = downtown_toronto_data

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,1,Coffee Shop,Park,Breakfast Spot,Performing Arts Venue,Pub,Restaurant,Spa,Bakery,Dessert Shop,Distribution Center
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,Yoga Studio,Burrito Place,Mexican Restaurant,Creperie,Beer Bar,Diner,Italian Restaurant,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Café,Shopping Mall,Steakhouse,Burrito Place,Mexican Restaurant,Clothing Store,Coffee Shop,Plaza,Ramen Restaurant,Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Gastropub,Café,Coffee Shop,BBQ Joint,Food Truck,Hotel,Italian Restaurant,Japanese Restaurant,Creperie,Cosmetics Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Seafood Restaurant,Bistro,Farmers Market,Bakery,Liquor Store,Breakfast Spot,Beer Bar,Concert Hall,Coffee Shop,Cocktail Bar


In [46]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters
Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.### Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 
Based on the defining categories, we can then assign a name to each cluster.

# Cluster 1 (Airport Lounge, Coffee Shop, Cafe, Restaurants & Grocery Store)

In [47]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Café,Shopping Mall,Steakhouse,Burrito Place,Mexican Restaurant,Clothing Store,Coffee Shop,Plaza,Ramen Restaurant,Restaurant
10,Downtown Toronto,0,Café,Coffee Shop,Bakery,Gastropub,Gym / Fitness Center,Ice Cream Shop,Japanese Restaurant,Deli / Bodega,Museum,Pub
11,Downtown Toronto,0,Bookstore,Restaurant,Bakery,Japanese Restaurant,Yoga Studio,Italian Restaurant,Café,College Gym,Comfort Food Restaurant,Sandwich Place


# Cluster 2 (Gastropubs)

In [48]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Park,Breakfast Spot,Performing Arts Venue,Pub,Restaurant,Spa,Bakery,Dessert Shop,Distribution Center
4,Downtown Toronto,1,Seafood Restaurant,Bistro,Farmers Market,Bakery,Liquor Store,Breakfast Spot,Beer Bar,Concert Hall,Coffee Shop,Cocktail Bar
8,Downtown Toronto,1,Park,Hotel,Plaza,Café,Performing Arts Venue,Bubble Tea Shop,Lake,Salad Place,Japanese Restaurant,Skating Rink
16,Downtown Toronto,1,Café,Restaurant,Park,Bakery,Gift Shop,Diner,Deli / Bodega,Indian Restaurant,Italian Restaurant,Japanese Restaurant
18,Downtown Toronto,1,Park,Diner,Juice Bar,Restaurant,Beer Bar,Ramen Restaurant,Bookstore,Italian Restaurant,Creperie,Dance Studio


# Cluster 3 (Cafes)


In [49]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Bar,Boat or Ferry,Sculpture Garden,Airport Gate


# Cluster 4 (Coffee Shop, Cafe, Park & Japanese Restaurant)


In [50]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,3,Gastropub,Café,Coffee Shop,BBQ Joint,Food Truck,Hotel,Italian Restaurant,Japanese Restaurant,Creperie,Cosmetics Shop
5,Downtown Toronto,3,Coffee Shop,Gastropub,Ice Cream Shop,Café,Park,Modern European Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Ramen Restaurant,Sandwich Place
6,Downtown Toronto,3,Grocery Store,Café,Park,Candy Store,Coffee Shop,Diner,Baby Store,Italian Restaurant,Restaurant,Nightclub
7,Downtown Toronto,3,Coffee Shop,Café,Bar,Seafood Restaurant,Plaza,Smoke Shop,Hotel,Speakeasy,Asian Restaurant,Steakhouse
9,Downtown Toronto,3,Café,Coffee Shop,Deli / Bodega,Gym,Hotel,Pub,Bookstore,Restaurant,Beer Bar,Japanese Restaurant
12,Downtown Toronto,3,Café,Vietnamese Restaurant,Mexican Restaurant,Wine Bar,Bakery,Fish Market,Dessert Shop,Coffee Shop,Cocktail Bar,Organic Grocery
14,Downtown Toronto,3,Park,Playground,Trail,Dance Studio,Caribbean Restaurant,Cheese Shop,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop
15,Downtown Toronto,3,Café,Cocktail Bar,Art Gallery,Tailor Shop,Restaurant,Beer Bar,Comfort Food Restaurant,Jazz Club,Museum,Hotel
17,Downtown Toronto,3,Café,Coffee Shop,Restaurant,Pizza Place,Bookstore,Deli / Bodega,Bakery,Steakhouse,Pub,Gym


# Cluster 5 (Seafood, steakhouse, Hotel & Cafe)

In [51]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,4,Coffee Shop,Sushi Restaurant,Yoga Studio,Burrito Place,Mexican Restaurant,Creperie,Beer Bar,Diner,Italian Restaurant,Distribution Center


# Exploring Neighborhoods in Manhattan


In [52]:
#Let's create a function to repeat the same process to all the neighborhoods in Manhattan
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'],
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [56]:
# Let's check how many venues were returned for each neighborhood
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,20,20,20,20,20,20
Carnegie Hill,20,20,20,20,20,20
Central Harlem,20,20,20,20,20,20
Chelsea,20,20,20,20,20,20
Chinatown,20,20,20,20,20,20
Civic Center,20,20,20,20,20,20
Clinton,20,20,20,20,20,20
East Harlem,20,20,20,20,20,20
East Village,20,20,20,20,20,20


In [59]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 211 uniques categories.


# Analyzing the Neighborhoods

In [60]:
#one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Outdoor Sculpture,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pub,Public Art,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Tiki Bar,Tourist Information Center,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [61]:
# Set Index
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

In [62]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0   Memorial Site  0.15
1            Park  0.15
2      Food Court  0.10
3          Garden  0.05
4  Cooking School  0.05


----Carnegie Hill----
                  venue  freq
0    Italian Restaurant  0.10
1  Gym / Fitness Center  0.10
2                   Gym  0.10
3                  Café  0.05
4           Coffee Shop  0.05


----Central Harlem----
                venue  freq
0  African Restaurant  0.10
1   French Restaurant  0.10
2           Jazz Club  0.05
3                 Bar  0.05
4        Dessert Shop  0.05


----Chelsea----
                 venue  freq
0       Ice Cream Shop  0.10
1   Seafood Restaurant  0.10
2              Theater  0.10
3             Beer Bar  0.05
4  Japanese Restaurant  0.05


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.10
1      Sandwich Place  0.10
2         Yoga Studio  0.05
3              Bakery  0.05
4              Museum  0.05


----Civic Center----
                   venue  freq

In [63]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Memorial Site,Park,Food Court,Building,Gym,Garden,Sandwich Place,Shopping Mall,Gourmet Shop,Auditorium
1,Carnegie Hill,Gym,Gym / Fitness Center,Italian Restaurant,Café,Gourmet Shop,Bookstore,Coffee Shop,Bagel Shop,Community Center,Bar
2,Central Harlem,French Restaurant,African Restaurant,Bagel Shop,Café,Music Venue,Boutique,Ethiopian Restaurant,Cocktail Bar,Library,Beer Bar
3,Chelsea,Ice Cream Shop,Seafood Restaurant,Theater,Japanese Restaurant,Cupcake Shop,Market,Scenic Lookout,Chinese Restaurant,Office,Taco Place
4,Chinatown,Sandwich Place,Chinese Restaurant,Yoga Studio,New American Restaurant,Roof Deck,Tea Room,Cocktail Bar,Bakery,Greek Restaurant,Museum
5,Civic Center,Spa,French Restaurant,Yoga Studio,Bar,Molecular Gastronomy Restaurant,Burrito Place,Falafel Restaurant,Cuban Restaurant,Furniture / Home Store,Sushi Restaurant
6,Clinton,Theater,Gym / Fitness Center,Hotel,Sporting Goods Shop,Pizza Place,Peruvian Restaurant,Comedy Club,Cocktail Bar,Café,Building
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Spanish Restaurant,Pharmacy,French Restaurant,Steakhouse,Street Art,Latin American Restaurant,Sandwich Place
8,East Village,Dessert Shop,Vietnamese Restaurant,Pet Café,Swiss Restaurant,Juice Bar,Dog Run,Bar,Coffee Shop,Bagel Shop,Japanese Restaurant
9,Financial District,Coffee Shop,Gym / Fitness Center,Falafel Restaurant,Café,Salad Place,Doctor's Office,Spa,Restaurant,Jewelry Store,Seafood Restaurant


# CLUSTERING NEIGHBORHOODS
Now we applied Machine Learning Technique “Clustering” to segment the neighborhoods in similar objects cluster. This will help to analyze from Tourist perspective and 
we can easily extract the Tourist places which are present on one of the clusters

# Manhattan

In [67]:
#Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 3, 0, 0, 0, 3, 2, 0, 0], dtype=int32)

In [68]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!


,Unnamed: 0,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,6,Manhattan,Marble Hill,40.876551,-73.910660,1,Coffee Shop,Gym,Yoga Studio,Tennis Stadium,American Restaurant,Bank,Department Store,Diner,Discount Store,Donut Shop
1,100,Manhattan,Chinatown,40.715618,-73.994279,0,Sandwich Place,Chinese Restaurant,Yoga Studio,New American Restaurant,Roof Deck,Tea Room,Cocktail Bar,Bakery,Greek Restaurant,Museum
2,101,Manhattan,Washington Heights,40.851903,-73.936900,3,Wine Shop,Park,Café,Breakfast Spot,Bakery,Pizza Place,Deli / Bodega,Ramen Restaurant,Restaurant,Coffee Shop
3,102,Manhattan,Inwood,40.867684,-73.921210,0,Wine Bar,Park,Café,Yoga Studio,Bakery,Deli / Bodega,Diner,Farmers Market,Mexican Restaurant,Spanish Restaurant
4,103,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Yoga Studio,Cocktail Bar,Mexican Restaurant,Wine Bar,Bakery,Smoke Shop,Historic Site,Café,Caribbean Restaurant,Mediterranean Restaurant


In [69]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# EXAMINE CLUSTERS
Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 
Based on the defining categories, we can then assign a name to each cluster.

#  Manhattan
Residential


In [72]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Manhattan,0,Sandwich Place,Chinese Restaurant,Yoga Studio,New American Restaurant,Roof Deck,Tea Room,Cocktail Bar,Bakery,Greek Restaurant,Museum
3,Manhattan,0,Wine Bar,Park,Café,Yoga Studio,Bakery,Deli / Bodega,Diner,Farmers Market,Mexican Restaurant,Spanish Restaurant
4,Manhattan,0,Yoga Studio,Cocktail Bar,Mexican Restaurant,Wine Bar,Bakery,Smoke Shop,Historic Site,Café,Caribbean Restaurant,Mediterranean Restaurant
5,Manhattan,0,Italian Restaurant,Juice Bar,Sushi Restaurant,Gastropub,Climbing Gym,Dumpling Restaurant,Coffee Shop,Lounge,Ramen Restaurant,Japanese Curry Restaurant
8,Manhattan,0,Hotel,French Restaurant,Jazz Club,Seafood Restaurant,Spa,Pet Store,Sandwich Place,Bookstore,Coffee Shop,Optical Shop
9,Manhattan,0,Wine Shop,Deli / Bodega,Hobby Shop,Gym / Fitness Center,Café,Liquor Store,Sandwich Place,Beer Store,Dog Run,Coffee Shop
11,Manhattan,0,Deli / Bodega,Residential Building (Apartment / Condo),Liquor Store,School,Soccer Field,Coffee Shop,Café,Outdoors & Recreation,Japanese Restaurant,Greek Restaurant
12,Manhattan,0,American Restaurant,Bakery,Italian Restaurant,Ice Cream Shop,Tiki Bar,Chinese Restaurant,Movie Theater,Seafood Restaurant,Mediterranean Restaurant,Southern / Soul Food Restaurant
13,Manhattan,0,Theater,Concert Hall,Performing Arts Venue,Indie Movie Theater,Gym / Fitness Center,Fountain,Opera House,Circus,Library,Gift Shop
19,Manhattan,0,Dessert Shop,Vietnamese Restaurant,Pet Café,Swiss Restaurant,Juice Bar,Dog Run,Bar,Coffee Shop,Bagel Shop,Japanese Restaurant


# Commercial Places¶

In [77]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,1,Coffee Shop,Gym,Yoga Studio,Tennis Stadium,American Restaurant,Bank,Department Store,Diner,Discount Store,Donut Shop
10,Manhattan,1,Thai Restaurant,Gym,Dessert Shop,Czech Restaurant,French Restaurant,Liquor Store,Salad Place,Taco Place,Restaurant,College Academic Building
15,Manhattan,1,Clothing Store,Plaza,French Restaurant,Cycle Studio,Park,Salad Place,Smoke Shop,Spa,Miscellaneous Shop,Food Truck
25,Manhattan,1,Pizza Place,Bar,Yoga Studio,Bakery,Hawaiian Restaurant,Italian Restaurant,Grocery Store,Korean Restaurant,Fried Chicken Joint,Park
32,Manhattan,1,Spa,French Restaurant,Yoga Studio,Bar,Molecular Gastronomy Restaurant,Burrito Place,Falafel Restaurant,Cuban Restaurant,Furniture / Home Store,Sushi Restaurant
33,Manhattan,1,Korean Restaurant,Japanese Restaurant,Gym / Fitness Center,Snack Place,Steakhouse,Fried Chicken Joint,Scenic Lookout,Cocktail Bar,Building,Coffee Shop
37,Manhattan,1,Park,Bar,Gym / Fitness Center,Fountain,Playground,Boat or Ferry,Gas Station,German Restaurant,Baseball Field,Cocktail Bar


# Tourist Areas & Hubs

In [78]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Manhattan,2,Mexican Restaurant,Thai Restaurant,Bakery,Spanish Restaurant,Pharmacy,French Restaurant,Steakhouse,Street Art,Latin American Restaurant,Sandwich Place
16,Manhattan,2,Burger Joint,Hotel,Coffee Shop,Jewish Restaurant,Grocery Store,Museum,New American Restaurant,Sandwich Place,Taco Place,Tea Room
18,Manhattan,2,Italian Restaurant,Sushi Restaurant,Yoga Studio,Gourmet Shop,Mediterranean Restaurant,Seafood Restaurant,French Restaurant,Caribbean Restaurant,Sandwich Place,New American Restaurant
23,Manhattan,2,Clothing Store,Arts & Crafts Store,Salon / Barbershop,Furniture / Home Store,Electronics Store,Italian Restaurant,Dessert Shop,Spanish Restaurant,Spa,Women's Store
31,Manhattan,2,Italian Restaurant,Grocery Store,Wine Shop,Coffee Shop,Rock Club,Sandwich Place,Ice Cream Shop,Mexican Restaurant,French Restaurant,Bar


# Center Acivity¶


In [79]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Manhattan,3,Wine Shop,Park,Café,Breakfast Spot,Bakery,Pizza Place,Deli / Bodega,Ramen Restaurant,Restaurant,Coffee Shop
6,Manhattan,3,French Restaurant,African Restaurant,Bagel Shop,Café,Music Venue,Boutique,Ethiopian Restaurant,Cocktail Bar,Library,Beer Bar
14,Manhattan,3,Theater,Gym / Fitness Center,Hotel,Sporting Goods Shop,Pizza Place,Peruvian Restaurant,Comedy Club,Cocktail Bar,Café,Building
17,Manhattan,3,Ice Cream Shop,Seafood Restaurant,Theater,Japanese Restaurant,Cupcake Shop,Market,Scenic Lookout,Chinese Restaurant,Office,Taco Place


# Cultural & Going Out Places

In [80]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Manhattan,4,Italian Restaurant,Cocktail Bar,Coffee Shop,Bakery,Boutique,Board Shop,Austrian Restaurant,Park,Gastropub,Hardware Store
28,Manhattan,4,Memorial Site,Park,Food Court,Building,Gym,Garden,Sandwich Place,Shopping Mall,Gourmet Shop,Auditorium
30,Manhattan,4,Gym,Gym / Fitness Center,Italian Restaurant,Café,Gourmet Shop,Bookstore,Coffee Shop,Bagel Shop,Community Center,Bar
34,Manhattan,4,Yoga Studio,Beer Garden,Grocery Store,Gym,Gym / Fitness Center,Thai Restaurant,Bagel Shop,Beer Store,Chinese Restaurant,Italian Restaurant
39,Manhattan,4,American Restaurant,Gym / Fitness Center,Residential Building (Apartment / Condo),Gym,Café,Music School,Supermarket,Furniture / Home Store,Cocktail Bar,Theater


# RESULTS
After clustering the data of the respective neighborhoods, both cities (Boroughs) have venues which can be explored and
attract the Tourists. The neighborhoods are much similar in features like Theaters, opera houses, food places, clubs, museums, parks etc. As far as concern to dissimilarity, it differs in terms of some unique places like historical places and monuments.
# Observations & Recommendations
When we compare the tourist places, we observe that the historical place is only situated in Downtown Toronto and the Monument or landmark venue is in Manhattan neighborhoods. Similarly, Airport facility, Harbor, Sculpture garden and Boat or ferry services are also available i**n Downtown Toronto while venues like Nightlife, Climbing gym and Museums are present in Manhattan.
As far as concern to recommendations, we recommend Downtown Toronto Neighborhoods will be considered first to visit. The tourists have an easily travelling access due to Airport facility, which not only saves time but also helps to save money. This saved money can be utilized to explore more, the attracting venues.
# Conclusion
The downtown Toronto and Manhattan neighborhoods have more like similar venues. As we know that every place is unique in its own way, so that’s argument is present in both neighborhoods. The dissimilarity exists in terms of some different venues and facilities but not on a larger extent.